### Statistics in Python

#### Data representation and interaction

##### Data as a table

The setting that we consider for statistical analysis is that of multiple observations or samples described
by a set of different attributes or features. The data can than be seen as a 2D table, or matrix, with
columns giving the different attributes of the data, and rows the observations. For instance, the data
contained in data/brain_size.csv:

##### The pandas data-frame

Creating dataframes: reading data files or converting arrays

Reading from a CSV file: Using the above CSV file that gives observations of brain size and weight
and IQ (Willerman et al. 1991), the data are a mixture of numerical and categorical values:

In [ ]:
import pandas

dados = pandas.read_csv('data/brain_size.csv', 
                 sep=';',
                 na_values=".")

dados

Creating from arrays: A pandas.DataFrame can also be seen as a dictionary of 1D ‘series’, eg arrays
or lists. If we have 3 numpy arrays:

In [ ]:
import numpy as np

t = np.linspace(-6, 6, 20)
sin_t = np.sin(t)
cos_t = np.cos(t)

We can expose them as a pandas.DataFrame:

In [ ]:
pandas.DataFrame({'t': t, 
                  'sin': sin_t,
                  'cos': cos_t})

Manipulating data

data is a pandas.DataFrame, that resembles R’s dataframe:

In [ ]:
dados.shape # 40 rows and 8 columns

In [ ]:
dados.columns # It has columns

In [ ]:

print(dados['Gender']) # Columns can be addressed by name

In [ ]:
# Simpler selector
dados[dados['Gender'] == 'Female']['VIQ'].mean()

groupby: splitting a dataframe on values of categorical variables:

In [ ]:
groupby_gender = dados.groupby('Gender')
for gender, value in groupby_gender['VIQ']:
    print((gender, value.mean()))

groupby_gender is a powerful object that exposes many operations on the resulting group of dataframes:

In [ ]:
groupby_gender.mean()

##### Plotting data

Pandas comes with some plotting tools (pandas.tools.plotting, using matplotlib behind the scene)
to display statistics of the data in dataframes:

Scatter matrices:

In [ ]:
from pandas.tools import plotting
plotting.scatter_matrix(data[['Weight', 'Height', 'MRI_Count']])

In [ ]:
plotting.scatter_matrix(data[['PIQ', 'VIQ', 'FSIQ']])

Two populations

The IQ metrics are bimodal, as if there are 2 sub-populations.

#### Hypothesis testing: comparing two groups

For simple statistical tests, we will use the scipy.stats sub-module of scipy:

In [ ]:
from scipy import stats

See also:
Scipy is a vast library. For a quick summary to the whole library, see the scipy chapter.

#### Student’s t-test: the simplest statistical test

1-sample t-test: testing the value of a population mean

scipy.stats.ttest_1samp() tests if the population mean of data is likely to be equal to a given value
(technically if observations are drawn from a Gaussian distributions of given population mean). It returns
the T statistic, and the p-value (see the function’s help):

In [ ]:
stats.ttest_1samp(data['VIQ'], 0)

2-sample t-test: testing for difference across populations

We have seen above that the mean VIQ in the male and female populations were different. To test if
this is significant, we do a 2-sample t-test with scipy.stats.ttest_ind():

In [ ]:
female_viq = data[data['Gender'] == 'Female']['VIQ']
male_viq = data[data['Gender'] == 'Male']['VIQ']
stats.ttest_ind(female_viq, male_viq)

##### Paired tests: repeated measurements on the same individuals

PIQ, VIQ, and FSIQ give 3 measures of IQ. Let us test
if FISQ and PIQ are significantly different. We can use a 2 sample test:

In [ ]:
stats.ttest_ind(data['FSIQ'], data['PIQ'])

The problem with this approach is that it forgets that there are links between observations: FSIQ
and PIQ are measured on the same individuals. Thus the variance due to inter-subject variability is
confounding, and can be removed, using a “paired test”, or “repeated measures test”:

In [ ]:
stats.ttest_rel(data['FSIQ'], data['PIQ'])

This is equivalent to a 1-sample test on the difference:

In [ ]:
stats.ttest_1samp(data['FSIQ'] - data['PIQ'], 0)

T-tests assume Gaussian errors. We can use a Wilcoxon signed-rank test, that relaxes this assumption:

In [ ]:
stats.wilcoxon(data['FSIQ'], data['PIQ'])

#### Linear models, multiple factors, and analysis of variance

##### “formulas” to specify statistical models in Python

A simple linear regression

First, we generate simulated data according to the model:

In [ ]:
import numpy as np

x = np.linspace(-5, 5, 20)
np.random.seed(1)

# normal distributed noise
y = -5 + 3*x + 4 * np.random.normal(size=x.shape)

# Create a data frame containing all the relevant variables
data = pandas.DataFrame({'x': x, 'y': y})

“formulas” for statistics in Python

Then we specify an OLS model and fit it:

In [ ]:
from statsmodels.formula.api import ols
model = ols("y ~ x", data).fit()

We can inspect the various statistics derived from the fit:

In [ ]:
print(model.summary())

Categorical variables: comparing groups or multiple categories

Let us go back the data on brain size:

In [ ]:
data = pandas.read_csv('examples/brain_size.csv', 
                       sep=';',
                       na_values=".")

We can write a comparison between IQ of male and female using a linear model:

In [ ]:
model = ols("VIQ ~ Gender + 1", data).fit()
print(model.summary())

In [ ]:
data_fisq = pandas.DataFrame({'iq': data['FSIQ'], 'type': 'fsiq'})
data_piq = pandas.DataFrame({'iq': data['PIQ'], 'type': 'piq'})
data_long = pandas.concat((data_fisq, data_piq))

print(data_long)

In [ ]:
model = ols("iq ~ type", data_long).fit()
print(model.summary())

In [ ]:
stats.ttest_ind(data['FSIQ'], data['PIQ'])

##### Multiple Regression: including multiple factors

In [ ]:
data = pandas.read_csv('examples/iris.csv')
model = ols('sepal_width ~ name + petal_length', data).fit()
print(model.summary())

##### Post-hoc hypothesis testing: analysis of variance (ANOVA)

In the above iris example, we wish to test if the petal length is different between versicolor and virginica,
after removing the effect of sepal width. This can be formulated as testing the difference between the
coefficient associated to versicolor and virginica in the linear model estimated above (it is an Analysis of
Variance, ANOVA). For this, we write a vector of ‘contrast’ on the parameters estimated: we want
to test "name[T.versicolor] - name[T.virginica]", with an F-test:

In [ ]:
print(model.f_test([0, 1, -1, 0]))

#### More visualization: seaborn for statistical exploration

Seaborn combines simple statistical fits with plotting on pandas dataframes.
Let us consider a data giving wages and many other personal information on 500 individuals (Berndt,
ER. The Practice of Econometrics. 1991. NY: Addison-Wesley).

In [ ]:
print(data)

##### Pairplot: scatter matrices

We can easily have an intuition on the interactions between continuous variables using seaborn.
pairplot() to display a scatter matrix:

In [ ]:
import seaborn

seaborn.pairplot(data,
                 vars=['WAGE', 'AGE', 'EDUCATION'],
                 kind='reg')

Categorical variables can be
plotted as the hue:

In [ ]:
seaborn.pairplot(data, 
                 vars=['WAGE', 'AGE', 'EDUCATION'],
                 kind='reg', hue='SEX')

---

Look and feel and matplotlib settings

Seaborn changes the default of matplotlib figures to achieve a more “modern”, “excel-like” look. It
does that upon import. You can reset the default using:

In [ ]:
from matplotlib import pyplot as plt
plt.rcdefaults()

Tip: To switch back to seaborn settings, or understand better styling in seaborn, see the relevent section of the seaborn documentation.

---

##### lmplot: plotting a univariate regression

A regression capturing the relation between one variable
and another, eg wage and eduction, can be plotted using seaborn.lmplot():

In [ ]:
seaborn.lmplot(y='WAGE', x='EDUCATION', data=data)

#### Testing for interactions

Do wages increase more with education
for males than females?

In [ ]:
result = sm.ols(formula='wage ~ education + gender + education * gender',
                data=data).fit()
print(result.summary())

#### Full code for the figures

##### Boxplots and paired differences

Plot boxplots for FSIQ, PIQ, and the paired difference between the two: while the spread (error bars)
for FSIQ and PIQ are very large, there is a systematic (common) effect due to the subjects. This effect
is cancelled out in the difference and the spread of the difference (“paired” by subject) is much smaller
than the spread of the individual measures.

In [ ]:
import pandas
import matplotlib.pyplot as plt

data = pandas.read_csv('brain_size.csv', sep=';', na_values='.')

# Box plot of FSIQ and PIQ (different measures od IQ)
plt.figure(figsize=(4, 3))
data.boxplot(column=['FSIQ', 'PIQ'])

# Boxplot of the difference
plt.figure(figsize=(4, 3))
plt.boxplot(data['FSIQ'] - data['PIQ'])
plt.xticks((1, ), ('FSIQ - PIQ', ))
plt.show()